# Preamble

In [1]:
import os, sys
import pandas as pd
import numpy as np

In [2]:
sys.path.append('/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/code/func/')
from proj_environment import set_proj_env
from func import get_synth_cov

In [3]:
train_test_str = 'squeakycleanExclude'
exclude_str = 't1Exclude' # 't1Exclude' 'fsFinalExclude'
parc_str = 'schaefer' # 'schaefer' 'lausanne'
parc_scale = 400 # 200 400 | 60 125
extra_str = ''
parcel_names, parcel_loc, drop_parcels, num_parcels, yeo_idx, yeo_labels = set_proj_env(train_test_str = train_test_str, exclude_str = exclude_str,
                                                                            parc_str = parc_str, parc_scale = parc_scale, extra_str = extra_str)

In [4]:
print(os.environ['MODELDIR_BASE'])
print(os.environ['MODELDIR'])

/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_400
/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_400


## Load data

In [5]:
# Load data
df = pd.read_csv(os.path.join(os.environ['MODELDIR_BASE'], 'df_pheno.csv'))
df.set_index(['bblid', 'scanid'], inplace = True)

df_node = pd.read_csv(os.path.join(os.environ['MODELDIR'], 'df_node_clean.csv'))
df_node.set_index(['bblid', 'scanid'], inplace = True)

# adjust sex to 0 and 1
df['sex_adj'] = df.sex - 1
print(df.shape)
print(df_node.shape)

(1099, 19)
(1099, 801)


In [6]:
df.head()

,,squeakycleanExclude,ageAtScan1,ageAtScan1_Years,sex,race2,handednessv2,dti64MeanAbsRMS,dti64MeanRelRMS,dti64MaxAbsRMS,dti64MaxRelRMS,mprage_antsCT_vol_TBV,averageManualRating,Overall_Psychopathology,Psychosis_Positive,Psychosis_NegativeDisorg,AnxiousMisery,Externalizing,Fear,sex_adj
bblid,scanid,,,,,,,,,,,,,,,,,,,
81287,2738,0,240,20.0,2,1,1,0.351665,0.154478,0.557358,0.272518,1.497389e+06,2.0,0.314394,-1.109349,-0.572468,1.222410,1.134094,-0.235063,1
80680,2739,1,253,21.1,1,1,1,0.531246,0.208078,0.754717,0.354560,1.813450e+06,2.0,0.748088,-0.905436,1.414109,0.032372,2.343126,0.744692,0
81754,2740,1,232,19.3,2,1,1,0.310943,0.218462,0.460633,0.387235,1.364954e+06,2.0,1.381240,0.986549,-0.969899,0.301802,-0.313214,-0.344655,1
81903,2749,0,231,19.2,2,1,1,0.432610,0.283153,0.819576,0.509537,1.266520e+06,2.0,0.531327,-0.885234,0.085093,1.994165,-0.831330,-1.902806,1
81043,2750,0,249,20.8,2,2,1,0.162409,0.096761,0.429102,0.259678,1.254653e+06,2.0,-0.330777,-0.788469,-0.798989,-2.036093,-0.072305,-0.746145,1


In [7]:
df_node.head()

,,squeakycleanExclude,ct_0,ct_1,ct_2,ct_3,ct_4,ct_5,ct_6,ct_7,ct_8,...,jd_390,jd_391,jd_392,jd_393,jd_394,jd_395,jd_396,jd_397,jd_398,jd_399
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
81287,2738,0,-0.426137,-0.176791,-0.234354,-0.083603,-0.384165,-0.185812,-0.462912,-0.539316,-0.496867,...,-0.074362,0.143993,0.324572,0.051980,0.146753,0.113563,0.193309,0.044411,0.010108,-0.013906
80680,2739,1,-0.153704,0.049472,-0.143747,-0.147157,0.207864,-0.115773,0.213060,-0.277668,0.274838,...,0.143000,0.186259,-0.185014,0.081583,-0.029763,0.114789,0.006170,0.298881,0.020025,0.182005
81754,2740,1,0.071090,0.567112,0.705114,0.411753,0.395432,0.358626,0.720517,-0.273845,0.130279,...,0.018973,-0.296800,-0.326918,-0.208356,-0.129417,-0.051025,-0.225684,-0.150662,-0.296544,-0.314343
81903,2749,0,0.022032,-0.035131,0.166311,-0.048539,-0.068253,-0.025678,-0.318749,0.340359,0.343013,...,0.074444,-0.032290,-0.250892,-0.105620,-0.237261,-0.034144,-0.234480,-0.092376,-0.332105,-0.204613
81043,2750,0,-0.271610,-0.608207,0.296959,-0.490972,-0.105437,-0.287931,-0.592220,0.519860,0.353512,...,-0.149667,-0.124684,-0.246320,-0.070710,-0.028870,0.103992,0.133622,0.085579,0.057805,-0.232649


# Prepare files for normative modelling

In [8]:
# Note, 'ageAtScan1_Years' is assumed to be covs[0] and 'sex_adj' is assumed to be covs[1]
# if more than 2 covs are to be used, append to the end and age/sex will be duplicated accordingly in the forward model
covs = ['ageAtScan1_Years', 'sex_adj']

print(covs)
num_covs = len(covs)
print(num_covs)

['ageAtScan1_Years', 'sex_adj']
2


In [9]:
extra_str_2 = ''

## Primary model (train/test split)

In [10]:
# Create subdirectory for specific normative model -- labeled according to parcellation/resolution choices and covariates
normativedir = os.path.join(os.environ['MODELDIR'], '+'.join(covs) + extra_str_2 + '/')
print(normativedir)
if not os.path.exists(normativedir): os.mkdir(normativedir);

/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_400/ageAtScan1_Years+sex_adj/


In [11]:
# Write out training -- retaining only residuals from nuissance regression
df[df[train_test_str] == 0].to_csv(os.path.join(normativedir, 'train.csv'))
df[df[train_test_str] == 0].to_csv(os.path.join(normativedir, 'cov_train.txt'), columns = covs, sep = ' ', index = False, header = False)

resp_train = df_node[df_node[train_test_str] == 0].drop(train_test_str, axis = 1)
mask = np.all(np.isnan(resp_train), axis = 1)
if np.any(mask): print("Warning: NaNs in response train")
resp_train.to_csv(os.path.join(normativedir, 'resp_train.csv'))
resp_train.to_csv(os.path.join(normativedir, 'resp_train.txt'), sep = ' ', index = False, header = False)

# Write out test -- retaining only residuals from nuissance regression
df[df[train_test_str] == 1].to_csv(os.path.join(normativedir, 'test.csv'))
df[df[train_test_str] == 1].to_csv(os.path.join(normativedir, 'cov_test.txt'), columns = covs, sep = ' ', index = False, header = False)

resp_test = df_node[df_node[train_test_str] == 1].drop(train_test_str, axis = 1)
mask = np.all(np.isnan(resp_test), axis = 1)
if np.any(mask): print("Warning: NaNs in response train")
resp_test.to_csv(os.path.join(normativedir, 'resp_test.csv'))
resp_test.to_csv(os.path.join(normativedir, 'resp_test.txt'), sep = ' ', index = False, header = False)

print(str(resp_train.shape[1]) + ' features written out for normative modeling')

800 features written out for normative modeling


### Forward variants

In [12]:
fwddir = os.path.join(normativedir, 'forward/')
if not os.path.exists(fwddir): os.mkdir(fwddir)

# Synthetic cov data
x = get_synth_cov(df, cov = 'ageAtScan1_Years', stp = 1)

if 'sex_adj' in covs:
    # Produce gender dummy variable for one repeat --> i.e., to account for two runs of ages, one per gender
    gender_synth = np.concatenate((np.ones(x.shape),np.zeros(x.shape)), axis = 0)

# concat
synth_cov = np.concatenate((np.matlib.repmat(x, 2, 1), np.matlib.repmat(gender_synth, 1, 1)), axis = 1)
print(synth_cov.shape)

# write out
np.savetxt(os.path.join(fwddir, 'synth_cov_test.txt'), synth_cov, delimiter = ' ', fmt = ['%.1f', '%.d'])

(30, 2)


### Permutation test | train and test | no blocks

In [13]:
# number of permutations
num_perms = 1000

# Set seed for reproducibility
np.random.seed(0)

for i in range(num_perms):
    permdir = os.path.join(normativedir, 'perm_all/perm_' + str(i))
    if not os.path.exists(permdir): os.makedirs(permdir)

    df_shuffed = df.copy()
    df_shuffed.loc[:,covs] = df_shuffed[covs].sample(frac = 1).values
    df_shuffed.loc[:,covs[1]] = df_shuffed.loc[:,covs[1]].astype(int)

    df_shuffed[df_shuffed[train_test_str] == 0].to_csv(os.path.join(permdir, 'cov_train.txt'), columns = covs, sep = ' ', index = False, header = False)
    df_shuffed[df_shuffed[train_test_str] == 1].to_csv(os.path.join(permdir, 'cov_test.txt'), columns = covs, sep = ' ', index = False, header = False)